In [33]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)

print("API Key Loaded Successfully" if openai_api_key else "API Key Not Found")

API Key Loaded Successfully


In [34]:
from pydantic import BaseModel, Field

class isWCNP(BaseModel):
    isWcnp: bool

In [39]:
identity_completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {
            "role": "system", 
             "content": "Identity wether user is asking about kubernetes or wcnp application. Do Note that kubernetes or Wcnp applications do have namespace and app"},
        {
            "role": "user",
            "content": "what is the status of iro-prod wcnp app of item-assembler namespace?",
        },
    ],
    response_format=isWCNP
)

In [40]:
wcnp = identity_completion.choices[0].message.parsed
wcnp.isWcnp

True

In [19]:
# 

In [41]:
class WCNP(BaseModel):
    namespace: str
    appId: str

In [45]:
wcnp_completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Extract the kubernetes application information."},
        {
            "role": "user",
            "content": "what is the status of iro-prod app of item-assembler namespace?",
        },
    ],
    response_format=WCNP
)

In [46]:
wcnp_app = wcnp_completion.choices[0].message.parsed
wcnp_app.namespace

'item-assembler'

In [47]:
wcnp_app.appId

'iro-prod'

In [48]:
# 

In [25]:
class Oneops(BaseModel):
    org: str
    assembly: str
    platform: str

In [26]:
oneops_completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Extract the oneops application information."},
        {
            "role": "user",
            "content": "what is the status for platform iro-prod deployed in item-assembler assembly that belongs to iro?",
        },
    ],
    response_format=Oneops
)

In [27]:
oneops_app = oneops_completion.choices[0].message.parsed
oneops_app.org

'iro'

In [28]:
oneops_app.assembly

'item-assembler'

In [29]:
oneops_app.platform

'iro-prod'

In [30]:
oneops_app

Oneops(org='iro', assembly='item-assembler', platform='iro-prod')

In [31]:
# 

In [62]:
import requests
import json

def get_oneops(org, assembly, platform):
    response = requests.post(
        "http://localhost:9099/wcnp-app/test-wcnp",
        json={"org": org, "assembly": assembly, "platform": platform}
    )
    data = response.json()
    return data

def get_wcnp(namespace, appId):
    response = requests.post(
        "http://localhost:9099/wcnp-app/test-wcnp",
        json={"namespace": namespace, "appId": appId}
    )
    data = response.json()
    return data

In [63]:
# 

In [64]:
wcnp_tool = [
    {
        "type": "function",
        "function": {
            "name": "get_wcnp",
            "description": "Get WCNP application details",
            "parameters": {
                "type": "object",
                "properties": {
                    "namespace": {"type": "string"},
                    "appId": {"type": "string"},
                },
                "required": ["namespace", "appId"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    }
]

In [65]:
system_prompt = "You are a helpful assistant."

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "what is the status of iro-prod app of item-assembler namespace?"},
]
            

wcnp_tool_completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=wcnp_tool,
)

In [66]:
wcnp_tool_completion.model_dump()

{'id': 'chatcmpl-BG6iyWeUnsejSKHkXuHZbK8e25YKv',
 'choices': [{'finish_reason': 'tool_calls',
   'index': 0,
   'logprobs': None,
   'message': {'content': None,
    'refusal': None,
    'role': 'assistant',
    'annotations': [],
    'audio': None,
    'function_call': None,
    'tool_calls': [{'id': 'call_pgHl2L0QrK8e4NDYU4vmw7OH',
      'function': {'arguments': '{"namespace":"item-assembler","appId":"iro-prod"}',
       'name': 'get_wcnp'},
      'type': 'function'}]}}],
 'created': 1743178796,
 'model': 'gpt-4o-2024-08-06',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_898ac29719',
 'usage': {'completion_tokens': 25,
  'prompt_tokens': 70,
  'total_tokens': 95,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}}

In [67]:
# 

In [ ]:
oneops_tool = [
    {
        "type": "function",
        "function": {
            "name": "get_oneops",
            "description": "Get WCNP application details",
            "parameters": {
                "type": "object",
                "properties": {
                    "org": {"type": "string"},
                    "assembly": {"type": "string"},
                    "platform": {"type": "string"},
                },
                "required": ["org", "assembly", "platform"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    }
]

In [ ]:
system_prompt = "You are a helpful assistant."

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "what is the status of iro-prod app of item-assembler namespace?"},
]
            

oneops_tool_completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=oneops_tool,
)

In [ ]:
oneops_tool_completion.model_dump()

In [54]:
# 

In [68]:
def call_function(name, args):
    if name == "get_wcnp":
        return get_wcnp(**args)
    elif name == "get_oneops":
        return get_oneops(**args)

In [69]:
for tool_call in wcnp_tool_completion.choices[0].message.tool_calls:
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    messages.append(wcnp_tool_completion.choices[0].message)

    result = call_function(name, args)
    messages.append(
        {"role": "tool", "tool_call_id": tool_call.id, "content": json.dumps(result)}
    )

In [78]:
class WcnpResponse(BaseModel):
    status: str = Field(
        description="status of wcnp application"
    )
    response: str = Field(
        description="A natural language response to the user's question."
    )

class OneopsResponse(BaseModel):
    status: str = Field(
        description="status of oneops application"
    )
    response: str = Field(
        description="A natural language response to the user's question."
    )

In [79]:
messages

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user',
  'content': 'what is the status of iro-prod app of item-assembler namespace?'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_pgHl2L0QrK8e4NDYU4vmw7OH', function=Function(arguments='{"namespace":"item-assembler","appId":"iro-prod"}', name='get_wcnp'), type='function')]),
 {'role': 'tool',
  'tool_call_id': 'call_pgHl2L0QrK8e4NDYU4vmw7OH',
  'content': '{"status": "iro-prod is deployed successfully in item-assembler"}'}]

In [80]:
final_wcnp_completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=messages,
    tools=wcnp_tool,
    response_format=WcnpResponse,
)

In [81]:
final_wcnp_completion.model_dump()

{'id': 'chatcmpl-BG6purfQYX6uBeIN2MvGqz6N6M8JD',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': '{"status":"iro-prod is deployed successfully in item-assembler","response":"The iro-prod app in the item-assembler namespace is currently deployed successfully."}',
    'refusal': None,
    'role': 'assistant',
    'annotations': [],
    'audio': None,
    'function_call': None,
    'tool_calls': None,
    'parsed': {'status': 'iro-prod is deployed successfully in item-assembler',
     'response': 'The iro-prod app in the item-assembler namespace is currently deployed successfully.'}}}],
 'created': 1743179226,
 'model': 'gpt-4o-2024-08-06',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_898ac29719',
 'usage': {'completion_tokens': 39,
  'prompt_tokens': 185,
  'total_tokens': 224,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rej

In [82]:
final_response = final_wcnp_completion.choices[0].message.parsed
final_response.status

'iro-prod is deployed successfully in item-assembler'

In [83]:
final_response.response

'The iro-prod app in the item-assembler namespace is currently deployed successfully.'